In [46]:
from __future__ import annotations
from typing import *

import metadsl as m
import metadsl_core as mc
import metadsl_visualize.visualize

metadsl_visualize.visualize.monkeypatch()

In [47]:
class Array(m.Expression):
    @m.expression
    @classmethod
    def create(cls) -> Array:
        ...

    @m.expression
    def __getitem__(self, idx: Union[int, slice]) -> Array:
        ...
    
    @m.expression
    def index_int(self, i: mc.Integer) -> Array:
        ...

    @m.expression
    def index_slice(self, start: mc.Integer, stop: mc.Integer, step: mc.Integer) -> Array:
        ...

    @m.expression
    @property
    def get_index(self) -> Union[slice, int]:
        ...

#     @m.expression
#     @property
#     def length(self) -> mc.Integer:
#         ...

In [51]:
(
        IPython.get_ipython()
        and IPython.get_ipython().__class__.__name__ == "ZMQInteractiveShell"
    )

True

In [50]:
IPython.get_ipython().__class__.__name__

'ZMQInteractiveShell'

In [40]:
@mc.register
@m.rule
def _getitem_typed(a: Array, i: int, s: slice):
    yield a[i], a.index_int(mc.Integer.from_int(i))
    yield a[s], lambda: a.index_slice(
        mc.Integer.from_int(s.start),
        mc.Integer.from_int(s.stop),
        mc.Integer.from_int(s.step if s.step is not None else 1)
    )

In [41]:
@mc.register
@m.rule
def _slice_of_slice(
    a: Array,

    i: mc.Integer,
    j: mc.Integer,
    k: mc.Integer,

    l: mc.Integer,
    m: mc.Integer,
    n: mc.Integer
):
    """    
    a[i:j:k][l:m:n] == a[i + j, i + m, n * k]
    """
    return (
        a.index_slice(i, j, k).index_slice(l, m, n),
        a.index_slice(i + j, i + m, n * k)
    )

In [42]:
@mc.register
@m.rule
def _get_index(i: int, j: int, k: int):
    yield Array.create().index_int(mc.Integer.from_int(i)).get_index, i
    yield Array.create().index_slice(
        mc.Integer.from_int(i),
        mc.Integer.from_int(j),
        mc.Integer.from_int(k),
    ).get_index, slice(i, j, k)

In [44]:
Array.create()[1:2:2][2:3:4].get_index

Typez(definitions=None, nodes=[CallNode(id='-308666362331542466', function='Array.create', type_params=None, args=None, kwargs=None), PrimitiveNode(id='-3312730341382343964', type='slice', repr='slice(1, 2, 2)'), PrimitiveNode(id='-6362854663408583544', type='slice', repr='slice(2, 3, 4)'), CallNode(id='70989689021931579', function='Array.__getitem__', type_params=None, args=['-308666362331542466', '-3312730341382343964'], kwargs=None), CallNode(id='4138529908789473228', function='Array.__getitem__', type_params=None, args=['70989689021931579', '-6362854663408583544'], kwargs=None), CallNode(id='1443599856298358970', function='Array.get_index', type_params=None, args=['4138529908789473228'], kwargs=None), PrimitiveNode(id='-1627799018779946948', type='int', repr='1'), PrimitiveNode(id='-1398281825510612154', type='int', repr='2'), CallNode(id='-230806222744429328', function='Integer.from_int', type_params=None, args=['-1627799018779946948'], kwargs=None), CallNode(id='-6782319812949040

slice(3, 4, 8)

In [31]:
# @mc.register
# @m.rule
# def _create_length(name: str, l: Optional[int], l_integer: mc.Integer):
#     def result():
#         if l is None:
#             raise m.NoMatch()
#         return Array.create_length(name, mc.Integer.from_int(l))

#     yield Array.create(name, l), result,
#     yield Array.create_length(name, l_integer).length, l_integer